In [1]:

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, concatenate_datasets
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')


# url = 'faisalq/bert-base-arabic-wordpiece'
url = 'faisalq/bert-base-arabic-senpiece'
# url = 'faisalq/bert-base-arabic-bbpe'

df = pd.read_csv('WikiFANE_2014_Gold_standard/WikiFANE_Gold_2014_500K.txt', sep='\t', encoding='utf-8', 
                 header=None, names=['Word', 'tok'], engine='python', quotechar="\x07")
df.fillna('', inplace=True)
df.loc[df['tok'] == '', 'tok'] = 'O'  # empty lines don't have a token which will acquire a new noise label
# display(df[35:50])

df['tok'] = df['tok'].astype('category')
df['label'] = df['tok'].cat.codes

# x = set(df['tok'].values)
# display(x)

num_classes = len(set(df['label'].values))
display(num_classes)

sentences = []
labels = []

current_sentence = []
current_labels = []

#convert the df into a format suitable for BERT
for _, row in df.iterrows():
    if row['Word'] != '':  #check if the word is not a sentence separator (period)
        current_sentence.append(row['Word'])
        current_labels.append(row['label'])
    else:
        sentences.append(current_sentence)
        labels.append(current_labels)        
        current_sentence = []
        current_labels = []


dfx = pd.DataFrame({
    'tokens': sentences,
    'ner_tags': labels
})

# print(dfx)

dataset = Dataset.from_pandas(dfx)

# display(dataset)



tokenizer = AutoTokenizer.from_pretrained(url)


model = AutoModelForTokenClassification.from_pretrained(url, 
                                num_labels=num_classes)

max_length = 128


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True,
                                 padding="max_length", max_length=max_length)
    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special token
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)  # Subword token
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs



# dataset = dataset['train']
dataset = dataset.train_test_split(test_size=0.2)

dataset_train = dataset['train']
dataset_validation = dataset['test']

display(len(dataset_train))
display(len(dataset_validation))

# display(dataset_train)
    
dataset_train = dataset_train.map(tokenize_and_align_labels, batched=True)
dataset_validation = dataset_validation.map(tokenize_and_align_labels, batched=True)





2024-02-05 22:06:51.096223: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-05 22:06:51.120046: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-05 22:06:51.561202: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


102

Some weights of BertForTokenClassification were not initialized from the model checkpoint at faisalq/bert-base-arabic-senpiece and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


12609

3153

Map:   0%|          | 0/12609 [00:00<?, ? examples/s]

Map:   0%|          | 0/3153 [00:00<?, ? examples/s]

In [2]:
from transformers import TrainingArguments, Trainer
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

os.environ["TOKENIZERS_PARALLELISM"] = "false"

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_predictions = [tokenizer.convert_ids_to_tokens(tp) for tp in true_predictions]
    true_labels = [tokenizer.convert_ids_to_tokens(tl) for tl in true_labels]

    # results = classification_report(true_labels, true_predictions)
    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
        # "report": results,
    }



epochs = 70
save_steps = 10000 #save checkpoint every 10000 steps
batch_size = 256



training_args = TrainingArguments(
    output_dir = 'bert_wp/',
    overwrite_output_dir=True,
    num_train_epochs = epochs,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    save_steps = save_steps,
    save_total_limit = 2, 
    fp16=True,
    learning_rate = 5e-5,  # 5e-5 is the default
    # weight_decay=0.01,
    logging_steps = 100, #50_000
    evaluation_strategy = 'steps',
    # logging_dir="logs.txt",
    eval_steps = 100
    
)




# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_validation,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()


Step,Training Loss,Validation Loss,Precision,Recall,F1
100,0.626200,0.223059,0.729853,0.703002,0.716176
200,0.145000,0.189296,0.758773,0.772539,0.765594
300,0.076400,0.194727,0.777006,0.790984,0.783933
400,0.044200,0.203093,0.779828,0.798666,0.789135
500,0.027200,0.226017,0.780389,0.802001,0.791048
600,0.017100,0.237474,0.785707,0.801193,0.793374
700,0.012400,0.248944,0.798004,0.795987,0.796994
800,0.008700,0.258943,0.788460,0.800334,0.794352
900,0.006700,0.260598,0.783421,0.805235,0.794178
1000,0.005200,0.267609,0.783257,0.809026,0.795933


{'eval_loss': 0.3256758451461792,
 'eval_precision': 0.7984209474315411,
 'eval_recall': 0.8074590661006671,
 'eval_f1': 0.8029145728643217,
 'eval_runtime': 2.2412,
 'eval_samples_per_second': 1406.862,
 'eval_steps_per_second': 5.801,
 'epoch': 70.0}

In [3]:
# 942553